In [14]:
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import torch
import pyro
from pyro.optim import Adam, ClippedAdam
import congas as cg
from congas.models import LatentCategorical
from pyro.infer import TraceMeanField_ELBO,TraceEnum_ELBO, TraceGraph_ELBO, Trace_ELBO


In [16]:
import pickle

data_file = open("data.pkl",'rb')

data = pickle.loads(data_file.read())
data_file.close()

param_file =  open("parameters.pkl",'rb')

param = pickle.loads(param_file.read())
param_file.close()


interface = cg.Interface(CUDA = False)

In [17]:
param

{'probs': tensor([0.2000, 0.6000, 0.0500, 0.0250, 0.0250]),
 'init_probs': 0.6,
 'purity': 0.48,
 'theta_shape_atac': tensor([12.4963, 16.6187,  4.3866,  9.7522]),
 'theta_rate_atac': tensor([0.0135, 0.0067, 0.0125, 0.0125]),
 'nb_size_init_atac': tensor([150., 150., 150., 150.]),
 'likelihood_atac': 'NB',
 'a': 0.1,
 'b': 100.0,
 'lambda': 0.0,
 'hidden_dim': 5,
 'binom_prior_limits': [40.0, 1000.0],
 'equal_sizes_sd': True,
 'Temperature': 20.0,
 'K': [1, 2, 3],
 'latent_type': 'G'}

In [18]:
I  = 15
sim_data = cg.simulate_data_congas(ncells = 5000, seed = 16, subclones=8, 
                                   mixture_proportion_rna = torch.ones(8) / 8,
                                  mixture_proportion_atac = torch.ones(8) / 8, n_segments=I)
sim_data

{'atac': {'exp': array([[  82.,  309.,  279., ...,  128.,   54.,   27.],
         [  38.,  238.,  293., ...,  120.,  187.,  117.],
         [ 330.,  178.,  122., ...,   79.,  294.,  201.],
         ...,
         [ 753., 1097.,  326., ...,  218.,  252., 1179.],
         [ 121.,  205.,  189., ...,  302.,  391.,  506.],
         [ 293.,  375.,  319., ...,  435.,  536.,  324.]]),
  'MAF': array([[0.37053729, 0.32253585, 0.4158179 , ..., 0.28930478, 0.40497724,
          0.2818496 ],
         [0.24322206, 0.39135757, 0.34969442, ..., 0.37679794, 0.37214288,
          0.36666653],
         [0.41908695, 0.38388334, 0.35205534, ..., 0.38393695, 0.37789072,
          0.22120664],
         ...,
         [0.39910356, 0.41747507, 0.47764789, ..., 0.3003187 , 0.23592494,
          0.28347963],
         [0.23674662, 0.33409717, 0.36924853, ..., 0.30981061, 0.33964248,
          0.19481125],
         [0.43433506, 0.31206319, 0.21098797, ..., 0.2784139 , 0.36360118,
          0.37149347]]),
  'labels'

In [19]:
sim_data["rna"]["exp"].shape

(5000, 15)

In [20]:
data = {
    "data_atac" : torch.tensor(sim_data["atac"]["exp"]).T.float(),
    "data_rna" : torch.tensor(sim_data["rna"]["exp"]).T.float(),
    "norm_factor_atac" : torch.tensor(sim_data["atac"]["norm_factors"]).float(),
    "norm_factor_rna" : torch.tensor(sim_data["rna"]["norm_factors"]).float(),
    "pld" : torch.tensor(sim_data['CNA_bulk']['total'].values).float(), 
}

In [21]:
param["binom_prior_limits"] = torch.tensor([10,1000])
param["theta_shape_atac"] = torch.tensor(sim_data["atac"]["theta_shape_atac"]).float()
param["theta_rate_atac"] = torch.tensor(sim_data["atac"]["theta_rate_atac"]).float()
param["theta_shape_rna"] = torch.tensor(sim_data["rna"]["theta_shape_rna"]).float()
param["theta_rate_rna"] = torch.tensor(sim_data["rna"]["theta_rate_rna"]).float()
param['nb_size_init_atac'] = torch.tensor([150., 150., 150., 150.])
param["K"] = 3
param['probs'] = torch.tensor([0.2000, 0.6000, 0.0500, 0.0250, 0.0250])
param["likelihood_atac"] = "NB"
param["likelihood_rna"] = "NB"
param["temperature"] = 10
param["lambda"] = 1
param["purity"] = None
param["nb_size_init_atac"] = torch.ones(I) * 150
param["nb_size_init_rna"] = torch.ones(I) * 150
param["CUDA"] = False
param["multiome"] = True



In [22]:
interface.set_model(LatentCategorical)
interface.set_optimizer(ClippedAdam)
interface.set_loss(Trace_ELBO)
interface.initialize_model(data)
interface.set_model_params(param)


In [23]:
ll = interface.run(steps = 20, param_optimizer = {"lr":0.1})



ELBO: 6.972898562  : 100%|██████████| 20/20 [00:01<00:00, 14.15it/s] 


Done!


In [24]:

import numpy as np


lr = interface.learned_parameters()
ICs = interface.calculate_ICs()


Computing assignment probabilities
Computing information criteria.


In [25]:
np.sum(lr["assignment_atac"] == 0)

1736

In [26]:
np.sum(lr["assignment_rna"] == 0)

1736